## Web Scraping 

https://consult.gov.scot/elections-and-constitutional-development-division/draft-referendum-bill/consultation/published_select_respondent

## Imports

In [17]:
# import sys
# print(sys.executable)
#! /opt/homebrew/Cellar/jupyterlab/3.4.8_1/libexec/bin/python3.11 -m pip install pandas

In [18]:
## Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Reading the file with the responses URLS

In [19]:
url_r_list = [line.strip() for line in open("../dataset/url_responses_list.txt", 'r')]
len(url_r_list)

6806

## Parsing the first URL of to get all the questions

In [20]:
radio_possible_answers={}
radio_possible_answers[1]="Individual"
radio_possible_answers[2]="Organisation"
radio_possible_answers[3]="Publish response with name"
radio_possible_answers[4]="Publish response only (without name)"
radio_possible_answers[5]="Do not publish response"


In [21]:
questions_total=['What is your name?','Are you responding as an individual or an organisation?',
 'The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:',
 'What are your views on the proposed arrangements for managing the referendum?',
 'What are your views on the proposed technical changes to polling and count arrangements?',
 'What are your views on the proposed changes to rules on permissible participants?']

In [22]:
# URL of the webpage
responses_id=[]
answers_total=[]
for i in range(0, 100):
    url = url_r_list[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the question and answer elements using appropriate HTML tags and class names
    question_elements = soup.find_all('h3', class_= "dss-heading-x-small dss-heading-bold")


    question_elements = soup.find_all('h3', class_= "dss-heading-x-small dss-heading-bold")
    #print(question_elements)

    questions=[]
    for row in question_elements:
        questions.append(row.get_text().strip())
    for i in range(0, len(questions)):
        q= questions[i].strip().replace("\n                                                ","")                                          
        char_str = '' .join((z for z in q if not z.isdigit()))
        questions[i]=char_str.replace(".","")
    
    
    no_answers=[]
    for q in range(0, len(questions_total)):
        if questions_total[q] not in questions:
            no_answers.append(q)
    #print("No_answers %s" % no_answers)
    
    # response id
    li = soup.find('div', class_= "dss-easy-reading")
    child = li.findChildren("h2" , recursive=False)
    respond_id=child[0].get_text().split("Response ")[1]
    responses_id.append(respond_id)
    
    # answers
    answer_elements = soup.find_all('div', class_= "cs-question-padding")

    #1. answer.name 
    answers=[]
    answer_elements = soup.find_all('div', class_= "cs-fake-text-input")
    try:
        answers.append(answer_elements[0].get_text().strip())
    except:
        answers.append('No Name')
    #print(answers)
    
    #2. Radio Buttons: Indiviual/Organization; Publish info Answers
    radio_answer_elements = soup.find_all('span', class_= "sr-only")

    radio_answers=[]
    for row in range(3, len(radio_answer_elements)):
        if "Ticked" in radio_answer_elements[row].get_text().strip():
            radio_answers.append(row-2)

    for i in radio_answers:
         answers.append(radio_possible_answers[i])
            
    # 4.5 Views    
    answers_elements = soup.find_all('div', class_= "cs-fake-textarea-input cs-fake-textarea-input-height-10")
    answers_tmp=[]
    for row in answers_elements:
        answers.append(row.get_text().strip())
    
  
    answers_final=[]

    a=0
    for k in range(0,5):
    
        if k in no_answers:
            answers_final.append("No Answered")
        else:
            #print(a)
            answers_final.append(answers[a])
            a=a+1
            
    
        
    answers_r=[respond_id] + answers_final
    answers_total.append(answers_r)



    
    


In [23]:
questions_total

['What is your name?',
 'Are you responding as an individual or an organisation?',
 'The Scottish Government would like your permission to publish your consultation response Please indicate your publishing preference:',
 'What are your views on the proposed arrangements for managing the referendum?',
 'What are your views on the proposed technical changes to polling and count arrangements?',
 'What are your views on the proposed changes to rules on permissible participants?']

In [24]:
header=["id"] + questions_total
print(len(header))

7


In [27]:
answers_total[0]

['633076343',
 'Fiona Smith',
 'Individual',
 'Publish response with name',
 'I have confidence in the polling arrangements and therefore am happy for them to contiinue . However, more scrutiny within the polling stations  woul alleviate the fears of others. Also more checks should be done that people voting are actually resident and not using family addresses.',
 'The  improvements seem sensible.']

In [25]:
for i in range(0, len(answers_total)):
    print(i, len(answers_total[i]))

0 6
1 6
2 6
3 6
4 6
5 6
6 6
7 6
8 6
9 6
10 6
11 6
12 6
13 6
14 6
15 6
16 6
17 6
18 6
19 6
20 6
21 6
22 6
23 6
24 6
25 6
26 6
27 6
28 6
29 6
30 6
31 6
32 6
33 6
34 6
35 6
36 6
37 6
38 6
39 6
40 6
41 6
42 6
43 6
44 6
45 6
46 6
47 6
48 6
49 6
50 6
51 6
52 6
53 6
54 6
55 6
56 6
57 6
58 6
59 6
60 6
61 6
62 6
63 6
64 6
65 6
66 6
67 6
68 6
69 6
70 6
71 6
72 6
73 6
74 6
75 6
76 6
77 6
78 6
79 6
80 6
81 6
82 6
83 6
84 6
85 6
86 6
87 6
88 6
89 6
90 6
91 6
92 6
93 6
94 6
95 6
96 6
97 6
98 6
99 6


## Creating a dataframe with the questions and answers

In [26]:
# Create the DataFrame
df = pd.DataFrame(answers_total, columns=header)

# Print the DataFrame
df

ValueError: 7 columns passed, passed data had 6 columns